# Lightweight Fine-Tuning Project

Describe your choices for each of the following

* PEFT technique: LORA
* Model: distilbert-base-uncased
* Evaluation approach: F1 and Accuracy
* Fine-tuning dataset: IMDB from HF

## Loading and Evaluating a Foundation Model

Load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [2]:
!pip install datasets

  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.65.0
    Uninstalling tqdm-4.65.0:
      Successfully uninstalled tqdm-4.65.0
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding

from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [5]:
model_name = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},
    label2id={"NEGATIVE": 0, "POSITIVE": 1}
)

# Move model to device
model = model.to(device)
print(model)

for param in model.parameters():
    param.requires_grad = True

c:\users\pouya\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\pouya\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For be

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [6]:
## load IMDB Dataset
dataset = load_dataset("imdb")

train_dataset = dataset["train"].shuffle(seed=42).select(range(3000))
test_dataset = dataset["test"].shuffle(seed=42).select(range(600))

c:\users\pouya\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\pouya\.cache\huggingface\hub\datasets--imdb. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating unsupervised split: 100%|██████████████████████████████████| 50000/50000 [00:00<00:00, 145739.09 examples/s]


In [7]:
def preprocessor(examples):
    return tokenizer(
        examples["text"], 
        padding="max_length", 
        truncation=True, 
        max_length=512
    )

In [8]:
tokenized_train = train_dataset.map(preprocessor, batched=True)
tokenized_test = test_dataset.map(preprocessor, batched=True)

Map: 100%|██████████████████████████████████████████████████████████████████| 600/600 [00:00<00:00, 1245.29 examples/s]


In [9]:
# Remove the text column as it's no longer needed
tokenized_train = tokenized_train.remove_columns(["text"])
tokenized_test = tokenized_test.remove_columns(["text"])

# Rename 'label' to 'labels' for the trainer
tokenized_train = tokenized_train.rename_column("label", "labels")
tokenized_test = tokenized_test.rename_column("label", "labels")

# Set format for PyTorch
tokenized_train.set_format("torch")
tokenized_test.set_format("torch")

In [12]:
from sklearn.metrics import f1_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean(), "f1": f1_score(labels, predictions)}

In [13]:
# Create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Evaluate the base model
print("\nEvaluating base model performance...")

eval_trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

eval_trainer.evaluate(eval_dataset=tokenized_test)


Evaluating base model performance...


C:\Users\pouya\AppData\Local\Temp\ipykernel_33112\2159945670.py:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  eval_trainer = Trainer(


{'eval_loss': 0.6996585726737976,
 'eval_model_preparation_time': 0.0025,
 'eval_accuracy': 0.51,
 'eval_f1': 0.0,
 'eval_runtime': 318.8407,
 'eval_samples_per_second': 1.882,
 'eval_steps_per_second': 0.235}

## Performing Parameter-Efficient Fine-Tuning

Create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [14]:
# Create LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=16,  # rank
    lora_alpha=32,  # LoRA scaling parameter
    lora_dropout=0.1,  # LoRA dropout
    bias="none",
    target_modules=["q_lin", "v_lin"],  # DistilBERT specific modules
)

# Create PEFT model
peft_model = get_peft_model(model, lora_config)

# Print the number of trainable parameters
peft_model.print_trainable_parameters()

trainable params: 887,042 || all params: 67,842,052 || trainable%: 1.3075


In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./tmp/",
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    push_to_hub=False,
)

# Create trainer
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Train the model
print("\nStarting PEFT training...")
trainer.train()


Starting PEFT training...


c:\users\pouya\venv\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\pouya\AppData\Local\Temp\ipykernel_33112\711867522.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
2025/05/29 13:43:01 ERROR mlflow.utils.async_logging.async_logging_queue: Run Id 2f478916ff424dcc85c87928d424bf13: Failed to log run data: Exception: Changing param values is not allowed. Param with key='output_dir' was already logged with value='tmp_trainer' for run ID='2f478916ff424dcc85c87928d424bf13'. Attempted logging new value './tmp/'.
2025/05/29 13:43:01 ERROR mlflow.utils.async_logging.async_logging_queue: Run Id 2f478916ff424dcc85c87928d424bf13: Failed to log run data: Exception: Changing param values is not allowed. Param with key='logg

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.289800,0.304616,0.866667,0.863481


In [ ]:
# Saving the model

peft_model.save_pretrained("/tmp/peft_lora_model")

## Performing Inference with a PEFT Model

Load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [ ]:
# Reloading the model
from peft import AutoPeftModelForSequenceClassification

reloaded_model = AutoPeftModelForSequenceClassification.from_pretrained("/tmp/peft_lora_model")
reloaded_model = reloaded_model.to(device)

In [ ]:
# Evaluate the PEFT model
peft_trainer = Trainer(
    model=reloaded_model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

peft_results = peft_trainer.evaluate(eval_dataset=tokenized_test)
peft_results